In [1]:
init_file = """/Users/julienkervizic/Downloads/maildir/benson-r/_sent_mail/1."""
init_dir = """/Users/julienkervizic/Downloads/maildir/"""

In [2]:
import email
import os, sys
import codecs

In [4]:
"""
    Reads through the Enron Corpus Email Directory
    Parse the emails and store them into a List of Dictionaries
"""
import sys
reload(sys)
sys.setdefaultencoding("utf-8")

emails = []
for root, dirs, files in os.walk(init_dir, topdown=False):
     for name in files:
            file_path = os.path.join(root, name)
            with codecs.open(file_path, "r",encoding='utf-8', errors='ignore') as f:
                read_data = f.read()
                import email
                email = email.message_from_string(read_data)
                emails.append(
                        {
                            'from': email['from'],
                            'to': email['to'],
                            'cc': email['cc'],
                            'bcc': email['bcc'],
                            'body': email.get_payload(decode=True)
                        }
                )

In [5]:
"""
    Preprocessor for Email inputs
    1) Only considers the new inputs providing by the email corespondence
    2) String replaces Emails, Links and Numbers
    3) Normalize text to lowercase
"""

def PreProcessor(email_dict): 
    #only take into account the original message
    om = "-----Original Message-----"
    email_dict['body'] = email_dict['body'].split(om)[0]
    email_dict['body'] = email_dict['body'].split("From:")[0]
    email_dict['body'] = email_dict['body'].split("To:")[0]
    
    import re
    # Getting rid of numbers
    email_dict['body'] = re.sub('([0-9]+)', ' ', email_dict['body'])
    # Replacing links
    email_dict['body'] = re.sub('(http[^\"|^\']+.)', ' ', email_dict['body'])
    email_dict['body'] = re.sub('(www\.+.)', '', email_dict['body'])
    # Replacing Emails 
    email_dict['body'] = re.sub('(\w+@\w+\.\w+)', ' ', email_dict['body'])
    # removing NONASCII
    from unidecode import unidecode
    try:
        email_dict['body'] = re.sub("""(<NONASCII>)""", ' ', email_dict['body'])
        email_dict['body'] =  email_dict['body'].encode("ascii")
    except: 
        None
    # lowercase all 
    email_dict['body'] = email_dict['body'].lower()
    return email_dict

In [6]:
def ValidEmail(email_dict):
    cond = []
    ## Excludes email forwarding 
    cond.append("""Forwarded by""" not in email_dict['body'])
    ## Remove HTML emails
    ## TODO
    return min(cond)

In [7]:
email_prepross = [
    PreProcessor(email) for email in emails
    if ValidEmail(email)
    ]

In [8]:
"""
    Only considers Email that have more than 20 non white space words
"""

def ValidPreprocessEmail(email_dict):
    cond = [] 
    cond.append(len([item for item in email_dict['body'].split(" ") if item != " "]) > 20)
    return min(cond)

validPreprocessed = [email for email in email_prepross if ValidPreprocessEmail(email)]

In [9]:
print "Total Emails", len(emails)
print "Emails with Original Message in body", len([email for email in emails if "Original Message" in email['body']])
print "Emails with Subject in Body", len([email for email in emails if "Subject:" in email['body']])
print "Forward Emails:", len([email for email in emails if "FW:" in email['body']])
print "Preprocessed Emails:", len(email_prepross)
print "Valid Preprocessed Emails for Training", len([email for email in email_prepross if ValidPreprocessEmail(email)])

In [10]:
import random
random.choice(validPreprocessed)['body']

'meeting to discuss the current information that we receive as well as the new \ngeneration database\n\nthe meeting will take place in  b'

In [11]:
import pickle
pickle.dump( validPreprocessed, open( "validPreprocessed.p", "wb" ) )

In [12]:
validPreprocessed = pickle.load(open( "validPreprocessed.p", "rb" ) )

In [13]:
def WordProcessor(input_text):
    return None

"""
    Tokenizes and Generate word Frequencies for a given text input
"""
def getFrequencies(input_text):
    import nltk
    from nltk.probability import FreqDist
    #nltk.download()
    tokens = nltk.word_tokenize(input_text)
    return FreqDist(tokens)

In [14]:
"""
    Obtains word frequencies in the Enron Email Corpus
"""

from nltk.probability import FreqDist
init_FreqDist = FreqDist()
for email in validPreprocessed:
    try:
        init_FreqDist.update(getFrequencies(email['body']))
    except:
        print("error", email['body'])
        print "unicode", isinstance(email['body'], unicode)
        print "str", isinstance(email['body'], str)
        

In [15]:
import pickle
pickle.dump( init_FreqDist, open( "init_FreqDist.p", "wb" ) )

In [16]:
import sys
reload(sys)  # Reload is a hack
sys.setdefaultencoding('UTF8')

"""
        Windsorize the dataset to exclude from consideration 
        The most frequent and infrequent 5% of words in the dataset
"""
num_words = len(init_FreqDist.most_common())
start_wcnt = num_words / 20
end_wcnt = num_words - start_wcnt
words_dataset = init_FreqDist.most_common()[start_wcnt:end_wcnt]

In [17]:
#words_dataset[:100]

In [18]:
isinstance(validPreprocessed[200]['body'], unicode)

False

In [19]:
pickle.dump( words_dataset, open( "words_dataset.p", "wb" ) )


In [ ]:
len(validPreprocessed)